In [19]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import json
import pandas as pd
import os 
from sklearn.metrics import r2_score # importing the regression model for plotting
import torch

import torch
import torch.nn as nn 
import torch.nn.functional as F # equivalent of keras.backend as k 
import torch.autograd as autograd 

In [20]:
def compute_invariants_uniaxial(lam): 
    # computes I1 and I2 for uniaxial tension/compression given stretch lambda
    # assumes incompressibility: lambda_2 = lambda_3 = lambda^(-1/2)

    I1 = lam**2 + 2 / lam 
    I2 = 2 * lam + 1 / lam**2
    return I1, I2

def dI1_dlam(lam):
    # dI1/d_lambda = 2_lambda - 2/(lambda^2)
    return 2 * lam - 2 / lam**2

def dI2_dlam(lam):
    # dI2/d_lambda = 2 - 2/lambda^3
    return 2 - 2 / lam**3

def compute_stress_uniaxial(lam, dPsi_dI1, dPsi_dI2): # note that the derivaties are passed later 
    # as a separate function they are calculated using autograd
    # Computes nominal stress P11 in uniaxial loading
    return 2 * (dPsi_dI1 + dPsi_dI2 / lam) * (lam - 1 / lam**2)

def compute_invariants_shear(gamma):
    # this is an invariant for simple shear: I1 = I2 = 3+gamma^2
    I1 = 3 + gamma**2
    I2 = 3 + gamma**2
    return I1, I2

def compute_stress_shear(gamma, dPsi_dI1, dPsi_dI2):
    # Compute nominal shear stress p12 in simple shear
    return  2 * gamma * (dPsi_dI1 + dPsi_dI2)

In [21]:
#Import excel file
file_name = 'input/CANNsBRAINdata.xlsx'
# dfs = pd.read_excel(file_name, sheet_name='Sheet1')
# Might need to install openpyxl to open excel files
dfs = pd.read_excel(file_name, sheet_name='Sheet1',engine='openpyxl')
# Load data for different brain regions
def getStressStrain(Region):
    if Region =='CX':
        P_ut = dfs.iloc[3:,1].dropna().astype(np.float64)
        lam_ut = dfs.iloc[3:,0].dropna().astype(np.float64)
        P_ss = dfs.iloc[3:,3].dropna().astype(np.float64).values
        gamma_ss = dfs.iloc[3:,2].dropna().astype(np.float64).values
    elif Region =='CR':
        P_ut = dfs.iloc[3:,6].dropna().astype(np.float64)
        lam_ut = dfs.iloc[3:,5].dropna().astype(np.float64)
        P_ss = dfs.iloc[3:,8].dropna().astype(np.float64).values
        gamma_ss = dfs.iloc[3:,7].dropna().astype(np.float64).values
    elif Region =='BG':
        P_ut = dfs.iloc[3:,11].dropna().astype(np.float64)
        lam_ut = dfs.iloc[3:,10].dropna().astype(np.float64)
        P_ss = dfs.iloc[3:,13].dropna().astype(np.float64).values
        gamma_ss = dfs.iloc[3:,12].dropna().astype(np.float64).values
    elif Region =='CC':
        P_ut = dfs.iloc[3:,16].dropna().astype(np.float64)
        lam_ut = dfs.iloc[3:,15].dropna().astype(np.float64)
        P_ss = dfs.iloc[3:,18].dropna().astype(np.float64).values
        gamma_ss = dfs.iloc[3:,17].dropna().astype(np.float64).values
        P_ut_all =P_ut
        lam_ut_all =lam_ut
    return P_ut_all, lam_ut_all, P_ut, lam_ut, P_ss, gamma_ss

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [22]:
# Define different loading protocols
def traindata(modelFit_mode):
    weighing_TC = np.array([0.5]*lam_ut[:16].shape[0] + 
                           [1.5]*lam_ut[16:].shape[0])  # weigh tension/compression differently

    if modelFit_mode == 'T':
        model_given = model_UT
        input_train = lam_ut[16:]
        output_train = P_ut[16:]
        sample_weights = np.array([1.0]*input_train.shape[0])

    elif modelFit_mode == "C":
        model_given = model_UT
        input_train = lam_ut[:17]
        output_train = P_ut[:17]
        sample_weights = np.array([1.0]*input_train.shape[0])

    elif modelFit_mode == "SS":
        model_given = model_SS
        input_train = gamma_ss
        output_train = P_ss
        sample_weights = np.array([1.0]*input_train.shape[0])

    elif modelFit_mode == "TC_and_SS":
        model_given = model
        input_train = [[lam_ut], [gamma_ss]]
        output_train = [[P_ut], [P_ss]]
        sample_weights_tc = weighing_TC
        sample_weights_ss = np.array([1.0]*gamma_ss.shape[0])
        sample_weights = [[sample_weights_tc], [sample_weights_ss]]

    return model_given, input_train, output_train, sample_weights

In [23]:
# The part below is a NN for the invariant based model  psi = psi(I1, I2)

# in the underformed configuration I1 = 3 and I2 = 3, therefore for the function they are passed as I-3

# I think they are using Neo-Hookean

In [24]:
def activation_exp(x):
    return torch.exp(x) - 1.0
    
def activation_ln(x):
    return -1.0 * torch.log(1.0 - x + 1e-7)

In [25]:
class SingleInvNet(nn.Module):
    def __init__(self, L2=0.0):
        super(SingleInvNet, self).__init__()
        self.linear_1 = nn.Linear(1, 1, bias = False)
        self.linear_2 = nn.Linear(1, 1, bias = False)
        self.linear_3 = nn.Linear(1, 1, bias = False)

        self.linear_4 = nn.Linear(1, 1, bias = False)
        self.linear_5 = nn.Linear(1, 1, bias = False)
        self.linear_6 = nn.Linear(1, 1, bias = False)

        #enforce non-negative weights 
        for layer in [self.linear_1, self.linear_2, self.linear_3, self.linear_4, self.linear_5, self.linear_6]:
            nn.init.uniform_(layer.weight, a = 0.0, b=0.1)

    def forward(self, I_ref):
        I_sq = I_ref**2

        out1 = self.linear_1(I_ref)
        out2 = activation_exp(self.linear_2(I_ref))
        out3 = activation_ln(self.linear_3(I_ref))

        out4 = self.lienar_4(I_sq)
        out5 = activation_exp(self.linear_5(I_sq))
        out6 = activation_ln(self.linear_6(I_sq))

        return torch.cat([out1, out2, out3, out4, out5, out6], dim = 1)

In [27]:
class StrainEneergyCANNInvariant(nn.Module):
    def __init__(self):
        super(StrainEnergyCANNInvariant, self).__init__()
        
        self.single_invent_I1 = SingleInvNet()
        self.single_invent_I2 = SingleInvNet()

        self.final_layer = nn.Linear(12, 1, bias = False) # combined 6 features from each block into one model
        nn.init.xavier_normal_(slef.final_layer.weight)

    def forward(self, I1, I2):
        I1_ref = I1 - 3.0
        I2_ref = I2 - 3.0

        out_I1 = self.single_invent_I1(I1_ref) # first batch of 6 
        out_I2 = self.single_invent_I2(I2_ref) # secong batch of 6

        combined = torch.cat([out_I1, out_I2], dim = 1)

        #compute strain energy 
        psi = self.final_layer(combined) 
        return psi

In [30]:
# next part computes stress from the strain energy psi
# it looks like they are using uniaxial tension/compression
# lamda in this case comes from the expiremental data that was loaded

In [31]:
# I1 = lambda^2 + 2*lambda^-1 
# I2 = 2*lambda + lambda^-2
# der_I1 = 2*lambda - 2*lambda^-2
# der_u2 = 2 - 2*lambda^-3

In [32]:
#imputs are derivative of psi over I1, derivative of psi over I2, lambda
def Stress_calc_TC(inputs):
    (dPsidI1, dPsidI2, Stretch) = inputs
    minus = 2 * (dPsidI1 / Stretch**2 + dPsidI2 / Stretch**3)
    stress = 2 * (dPsidI1 * Stretch + dPsidI2) - minus
    return stress

In [33]:
def Stress_cal_SS(inputs):
    (dPsidI1, dPsidI2, gamma) = inputs
    stress = 2 * gamma * (dPsidI1 + dPsidI2)
    return stress

In [34]:
# calculates the gradient where a is psi and b is either I1 or I2

In [36]:
def myGradients(a, b):
    return torch.autograd.grad(a, b, grad_outputs = torch.ones_like(a),
                               retain_graph=True, create_graph = True)[0]


In [37]:
# in the following calculation, keep in mind that stretch and shear comes from the expiremental data

In [ ]:
class StressModelAcrhitecture(nn.Module):
    def __init__(self, psi_model):
        super(StressModelArchitecture, self).__init__()
        self.psi_midel = psi_model # from the StrainENERGYCANNInvariant 

    def compute_invariants_ut(self, stretch):
        I1 = stretch**2 + 2.0 / stretch
        I2 = 2.0 * stretch + 1.0 / stretch**2
        return I1, I2

    def compute_invariants_ss(self, gamma):
        I1 = gamma**2 + 3.0
        I2 = gamma**2 + 3.0
        return I1, I2

    def forward(self, stretch, gamma):
        I1_ut, I2_ut = self.compute_invaraints_ut(stretch)
        I1_ut.requires_grad_(True)
        I2_ut.requires_grad_(True)

        psi_ut = self.psi_model(I1_ut, I2_ut)
        dpsi_dI1_ut = torch.autograd.grad(psi_ut, I1_ut, grad_outputs=torch.ones_like(psi_ut), create_graph=True)[0]
        dpsi_dI2_ut = torch.autograd.grad(psi_ut, I2_ut, grad_outputs=torch.ones_like(psi_ut), create_graph=True)[0]
        stress_ut = 2 * (dpsi_dI1_ut * stretch + dpsi_dI2_ut) - 2 * (dpsi_dI1_ut / stretch**2 + dpsi_dI2_ut / stretch**3)


        I1_ss, I2_ss = self.compute_invariants_ss(gamma)
        I1_ss.requires_grad_(True)
        I2_ss.requires_grad_(True)

        psi_ss = self.psi_model(I1_ss, I2_ss)
        dpsi_dI1_ss = torch.autograd.grad(psi_ss, I1_ss, grad_outputs=torch.ones_like(psi_ss), create_graph=True)[0]
        dpsi_dI2_ss = torch.autograd.grad(psi_ss, I2_ss, grad_outputs=torch.ones_like(psi_ss), create_graph=True)[0]
        stress_ss = 2 * gamma * (dpsi_dI1_ss + dpsi_dI2_ss)

        return stress_ut, stress_ss